In [1]:
import cv2
import glob
import os
from tqdm.notebook import tqdm
import numpy as np
import ffmpeg

In [2]:
file_paths = glob.glob("../Clips_small_size/*.mov")
file_paths = sorted(file_paths)

In [3]:
movie_info = {}

In [4]:
ratio = 0.8
patch_size = 16

In [5]:
for file_path in tqdm(file_paths):
    # get movie data path
    movie_name = os.path.splitext(os.path.basename(file_path))[0]
    print(movie_name)
    movie_info[movie_name] = {}

    # get movie data
    cap = cv2.VideoCapture(file_path)
    metadata = ffmpeg.probe(file_path)

    # save movie data
    width = int(metadata['streams'][1]['width'])
    height = int(metadata['streams'][1]['height'])
    size = np.array([width, height])
    resize = (np.ceil(size*ratio/patch_size) * patch_size).astype(int)
    movie_info[movie_name]["size"] = size
    movie_info[movie_name]["resize"] = resize
    movie_info[movie_name]["duration"] = float(metadata['streams'][1]['duration'])
    #movie_info[movie_name]["num_frames"] = int(metadata['streams'][1]['nb_frames'])
    movie_info[movie_name]["fps"] = eval(metadata['streams'][1]['avg_frame_rate'])
    print(movie_info[movie_name])
    
    # make save directory
    save_dir = f"../preprocessed_data/clips_png/{movie_name}"
    os.makedirs(save_dir, exist_ok=True)

    # save frames
    count = 0
    while True:
        ret, frame = cap.read()
        if ret == True:
            count += 1
            frame_resized = cv2.resize(frame, tuple(resize), interpolation=cv2.INTER_AREA)
            cv2.imwrite(save_dir + '/{:04d}.png'.format(count), frame_resized)
            #cv2.imwrite(save_dir + '/{:04d}.png'.format(count), frame)
        else:
            movie_info[movie_name]["num_frames"] = count
            print(count)
            break

  0%|          | 0/209 [00:00<?, ?it/s]

ADVEN_12a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.046667, 'fps': 23.962715789247916}
720
ADVEN_15a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.011667, 'fps': 23.989373595992294}
720
ADVEN_18a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.011667, 'fps': 23.989355833967004}
720
ADVEN_24a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.011667, 'fps': 23.956055021553414}
719
ADVEN_3a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.03, 'fps': 23.976015104889516}
720
ADVEN_9a_c
{'size': array([851, 468]), 'resize': array([688, 384]), 'duration': 30.03, 'fps': 23.976023976023978}
720
AMAZI_14a_c
{'size': array([851, 472]), 'resize': array([688, 384]), 'duration': 30.011667, 'fps': 23.98936471497636}
720
AMAZI_20a_c
{'size': array([851, 472]), 'resize': array([688, 384]), 'duration': 30.046667, 'fps': 23.929443088528956}
719
AMAZI_2a_c
{'size': array([

In [6]:
np.savez_compressed("../preprocessed_data/clips_png/movie_info.npz", **movie_info)